# Routing Settings
This module is for keeping the routing network synced across the `bikewaysim_framework`, `impedance_calibration`, and `map_matching` modules.

In [16]:
from pathlib import Path
import geopandas as gpd
import pickle
import pandas as pd

from bikewaysim.paths import config
from bikewaysim.network import prepare_network, modeling_turns

In [17]:
directed_links = pd.read_parquet(config['network_fp']/'directed_edges.parquet') # has the directional variables
links = pd.read_pickle(config['network_fp']/'final_network_edges.pkl') # non-directed
nodes = gpd.read_file(config['network_fp']/'final_network.gpkg',layer='nodes')
nodes = dict(zip(nodes['N'],nodes.geometry))
turns = pd.read_parquet(config['network_fp']/'turns_df.parquet')

In [18]:
links.columns

Index(['A', 'B', 'linkid', 'osmid', 'oneway', 'highway', 'name', 'all_tags',
       'link_type', 'facility_fwd', 'facility_rev', 'facility', 'year',
       'state_route', 'AADT', 'speed', 'lanes', 'ascent_ft', 'descent_ft',
       'ascent_grade_cat', 'descent_grade_cat', 'length_ft', 'geometry',
       '1lpd', '2lpd', '3+lpd', '[0,30] mph', '(30,40] mph', '(40,inf) mph',
       '(30,inf) mph', '[0,4k) aadt', '[4k,10k) aadt', '[10k,inf) aadt',
       '[0,4) grade', '[4,6) grade', '[6,inf) grade', 'bike lane',
       'cycletrack', 'multi use path', 'multi use path and cycletrack',
       'not_street'],
      dtype='object')

In [19]:
#add attributes back to links?
link_cols_drop = ['A','B','ascent_ft','ascent_grade_cat','descent_ft','descent_grade_cat','facility_fwd','facility_rev'] # drop directional attributes
links.drop(columns=link_cols_drop,inplace=True)
directed_cols_to_add = ['linkid','A','B','reverse_link','ascent_ft','ascent_grade_cat','facility_fwd']
links = pd.merge(links,directed_links[directed_cols_to_add],on='linkid')

In [20]:
def remove_wrongway_turns(turns_df,exceptions):
    """
    Remove turns that are not possible based on the link directionality.
    Can provide list of expections to keep.

    Exceptions provided in (linkid,reverse_link) format
    e.g. [(12345,True),(67890,False)]
    """

    # Remove wrongway turns unlesss they are in the map matching links
    oneway_dict = dict(zip(links['linkid'],links['oneway']))
    turns['source_oneway'] = turns['source_linkid'].map(oneway_dict)
    turns['target_oneway'] = turns['target_linkid'].map(oneway_dict)
    del oneway_dict

    source_exception = [(linkid,reverse_link) in exceptions for linkid, reverse_link in turns[['source_linkid','source_reverse_link']].values]
    target_exception = [(linkid,reverse_link) in exceptions for linkid, reverse_link in turns[['target_linkid','target_reverse_link']].values]
    # source_wrongway = ((turns['source_oneway'] == True) & (turns['source_reverse_link'] == True)) == False
    # target_wrongway = ((turns['target_oneway'] == True) & (turns['target_reverse_link'] == True)) == False
    source_wrongway = ((turns[['source_oneway','source_reverse_link']] == True).all(axis=1)==False) | (source_exception)
    target_wrongway = ((turns[['target_oneway','target_reverse_link']] == True).all(axis=1)==False) | (target_exception)
    turns = turns[source_wrongway & target_wrongway]

    #remove wrongway links
    #TODO did we remove these in the export network step too?
    exception = [(linkid,reverse_link) in exceptions for linkid, reverse_link in links[['linkid','reverse_link']].values]
    links = links.loc[((links[['oneway','reverse_link']]==True).all(axis=1) == False) | exception]

    return turns_df

In [21]:
# calculate travel time based on assumed speed
assumed_speed_mph = 9
links['travel_time_min'] = (links.length / 5280 / assumed_speed_mph * 60).round(8)

# only allow these types for routing
link_types_allowed = ['bike','pedestrian','road']
links = links[links['link_type'].isin(link_types_allowed)]

# remove isolated links
links, turns = prepare_network.remove_isolates(links,turns)
print(links.shape[0],'links and',len(set(pd.concat([links['A'],links['B']]).tolist())),'nodes')
print(turns.shape[0],'turns')

Before connected components: Links 27282 Nodes 12203
After connected components: Links 24116 Nodes 10473
Before connected components: Turns 164032
After connected components: Turns 36970
24116 links and 10473 nodes
36970 turns


In [22]:
#export calibration network
with (config['calibration_fp']/"calibration_network.pkl").open('wb') as fh:
    pickle.dump((links,turns),fh)